In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packag

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

In [15]:
# Create the vine_table DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show(5)
vine_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3EFW2STIYIY0I|          1|            2|          2|   N|                Y|
|R12NR0R5A9F7FT|          5|            0|          0|   N|                Y|
|R1LSH74R9XAP59|          2|            0|          1|   N|                Y|
|R1QXUNTF76K7L6|          2|            0|          0|   N|                Y|
|R2F7DR75PS8NKT|          5|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



341931

In [16]:
# Filter DataFrame to retrieve all rows where total_votes count is greater than or equal to 20
filtered_df = vine_df.filter("total_votes>=20").orderBy(vine_df["total_votes"].asc())
filtered_df.show()
filtered_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3C3UU3AWSPCPQ|          1|           13|         20|   N|                N|
|R1VZNSTRV9IE38|          1|           20|         20|   N|                Y|
|R2M9V2XMOA5MEJ|          1|           19|         20|   N|                Y|
|R1QMF01BDRT2LJ|          1|           10|         20|   N|                N|
|R2G908XLS4JISS|          5|           20|         20|   N|                N|
|R3E9P8SZEMFTBK|          2|           12|         20|   N|                N|
|R3K2ZVAQ2O3LWT|          1|            7|         20|   N|                Y|
|R1CXK4SGLJB4FE|          1|           15|         20|   N|                Y|
|R3GJGFTSDNVCHH|          5|           16|         20|   N|                N|
|R3JANDU5B4OSF0|          2|            6|         20|   N|     

20216

In [17]:
# Filter filtered_df to retrieve rows where helpful_votes/total_votes is greater than or equal to 50%
helpful_df = filtered_df.filter((filtered_df['helpful_votes']) / (filtered_df['total_votes']) >= 0.5)
helpful_df.show()
helpful_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R17A3UL7L22B8B|          3|           18|         20|   N|                N|
|R1QMF01BDRT2LJ|          1|           10|         20|   N|                N|
|R1FWWYZ2U0ZPJV|          1|           16|         20|   N|                Y|
|R3E9P8SZEMFTBK|          2|           12|         20|   N|                N|
|R1PXUULGJ2ARI8|          3|           20|         20|   N|                Y|
|R3GJGFTSDNVCHH|          5|           16|         20|   N|                N|
| R6A9T3U65BCUE|          1|           11|         20|   N|                N|
|R3QDRWZTGQPEDB|          1|           18|         20|   N|                Y|
|R1CXK4SGLJB4FE|          1|           15|         20|   N|                Y|
| RMXKAXWG5CDRN|          1|           16|         20|   N|     

17762

# Vine Program Analysis

In [18]:
# Filter helpful_df to create table that only has the rows where a review was written as part of vine program(paid)
vine_program_df = helpful_df.filter(helpful_df['vine'] == 'Y')
vine_program_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1BGEGOX6SCYU2|          5|           19|         20|   Y|                N|
| RWQZCIUS2Z8IS|          5|           19|         20|   Y|                N|
|R3BCPKHRR5942R|          3|           19|         20|   Y|                N|
|R2ODPCF6F4LT3O|          3|           17|         20|   Y|                N|
|R2SLQR7YTQELLA|          5|           19|         20|   Y|                N|
| RPQBEIHH92962|          4|           17|         20|   Y|                N|
|R3SF7YZ9N8DVN2|          5|           19|         20|   Y|                N|
|R3125W32EF7TY8|          5|           20|         20|   Y|                N|
| RSWJ1R6T04O02|          5|           19|         20|   Y|                N|
|R3TCHJ0QACS445|          4|           19|         20|   Y|     

In [19]:
# Total number of vine program reviews
total_vine_reviews = vine_program_df.count()
print(total_vine_reviews)

248


In [20]:
# 5-star reviews from vine program
five_star_vine_df = vine_program_df.filter(vine_program_df['star_rating'] == '5')
five_star_vine_df.show()

# 5-star review count from vine program
five_star_vine_reviews = five_star_vine_df.count()
print(five_star_vine_reviews)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R254QE8B67FICR|          5|           18|         20|   Y|                N|
|R2GSZW8LSS382H|          5|           17|         20|   Y|                N|
|R1BGEGOX6SCYU2|          5|           19|         20|   Y|                N|
| RWQZCIUS2Z8IS|          5|           19|         20|   Y|                N|
| RSWJ1R6T04O02|          5|           19|         20|   Y|                N|
|R2SLQR7YTQELLA|          5|           19|         20|   Y|                N|
| RW9U7VT1GELNS|          5|           16|         20|   Y|                N|
|R3125W32EF7TY8|          5|           20|         20|   Y|                N|
|R3SF7YZ9N8DVN2|          5|           19|         20|   Y|                N|
|R1AS6E12TGTGVP|          5|           21|         21|   Y|     

In [21]:
# Percentage of 5-star reviews from vine program
percentage = (five_star_vine_reviews / total_vine_reviews) * 100
print(round(percentage, 2))

41.13


# Non-Vine Program Analysis

In [22]:
# Filter helpful_df to create table that only has the rows where a review was not written as part of vine program(unpaid)
non_vine_program_df = helpful_df.filter(helpful_df['vine'] == 'N')
non_vine_program_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2G908XLS4JISS|          5|           20|         20|   N|                N|
| RF553TNOT43HO|          1|           19|         20|   N|                N|
|R3GJGFTSDNVCHH|          5|           16|         20|   N|                N|
|R17A3UL7L22B8B|          3|           18|         20|   N|                N|
| R1Q9Q3H7K7V90|          4|           19|         20|   N|                Y|
|R3E9P8SZEMFTBK|          2|           12|         20|   N|                N|
|R3R3MI19D1KJDF|          4|           20|         20|   N|                Y|
|R1QMF01BDRT2LJ|          1|           10|         20|   N|                N|
|R30TCX1Q46IFBX|          2|           20|         20|   N|                N|
|R3QDRWZTGQPEDB|          1|           18|         20|   N|     

In [23]:
# Total number of non-vine program reviews
total_non_vine_reviews = non_vine_program_df.count()
print(total_non_vine_reviews)

17514


In [24]:
# 5-star reviews from non-vine program
five_star_non_vine_df = non_vine_program_df.filter(non_vine_program_df['star_rating'] == '5')
five_star_non_vine_df.show()

# 5-star review count from vine program
five_star_non_vine_reviews = five_star_non_vine_df.count()
print(five_star_non_vine_reviews)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R7K9Q460DWPJS|          5|           13|         20|   N|                Y|
|R1S301EF3ODDBM|          5|           19|         20|   N|                Y|
|R1SAJANNCMGJIO|          5|           18|         20|   N|                Y|
|R2G908XLS4JISS|          5|           20|         20|   N|                N|
| RRJ1UYGM7BJIM|          5|           18|         20|   N|                N|
|R3GJGFTSDNVCHH|          5|           16|         20|   N|                N|
|R2GY7LXKR1W0G6|          5|           17|         20|   N|                Y|
|R26686YB558ZHX|          5|           19|         20|   N|                N|
|R1YP5ZQ1HZHRRI|          5|           20|         20|   N|                N|
|R18JQZB8Q0HJVH|          5|           20|         20|   N|     

In [25]:
# Percentage of 5-star reviews from non-vine program
percentage = (five_star_non_vine_reviews / total_non_vine_reviews) * 100
print(round(percentage, 2))

29.43
